# Setup

Start by importing TF Encrypted. We don't need to import TensorFlow as well but it's often very convenient since we can mix ordinary and encrypted computations.

In [1]:
import tensorflow as tf
import tf_encrypted as tfe

We only need the following step since we want to inspect the computation later in TensorBoard. It should normally be skipped to avoid the implied overhead of generating event and tracing files.

In [2]:
%load_ext tensorboard.notebook

TENSORBOARD_DIR = "/tmp/tensorboard"

tfe.setTFETraceFlag(True)
tfe.setMonitorStatsFlag(True)
tfe.setLogDirectory(TENSORBOARD_DIR)

INFO:tf_encrypted:Writing trace files for every session.run() call with a tag
INFO:tf_encrypted:Writing event files for every session.run() call with a tag
INFO:tf_encrypted:Writing event and trace files to '/tmp/tensorboard'


# Computation

We next define our mixed computation, in this case summing two encrypted (i.e. private) tensors coming from different input providers. Note that we are using ordinary TensorFlow to generate the inputs locally on the providers.

In [3]:
x = tfe.define_private_input("input-provider-x", lambda: tf.fill([2,2], 2))
y = tfe.define_private_input("input-provider-y", lambda: tf.fill([2,2], 3))

z = x + y

At this point `z` contains the encrypted sum. To reveal only the sum to a result receiver you would normally use the following, which decrypts and executes the print function locally on the result receiver:

In [4]:
# compute_op = tfe.define_output("result-receiver", z, tf.print)

However, since we are running in a notebook the above wouldn't actually display anything. To get around this we can use the following `print_in_notebook` function instead.

We stress that this is only because we are running in a notebook, and using `py_func` is for instance not possible when running in an actual distributed execution context. See the [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/print) for more information.

In [5]:
def print_in_notebook(x):
    return tf.py_func(print, [x], Tout=[])

compute_op = tfe.define_output("result-receiver", z, print_in_notebook)

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


# Execution

Having defined our computation we use a `tfe.Session` to run it, optionally passing in a tag when we want event and tracing files to be written.

Here we first remove previous event and tracing files to make it easier to find the new runs in TensorBoard.

In [6]:
!rm -rf {TENSORBOARD_DIR}

with tfe.Session() as sess:
    sess.run(compute_op, tag='sum')
    sess.run(compute_op, tag='sum')

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2', 'input-provider-x', 'input-provider-y', 'result-receiver']


[[5. 5.]
 [5. 5.]]
[[5. 5.]
 [5. 5.]]


# Inspection

We can finally inspect our computations in TensorBoard using the tags passed to `sess.run`.

Note that this is not saved in notebooks so nothing will show below unless you run this yourself.

In [7]:
%tensorboard --logdir {TENSORBOARD_DIR}

Reusing TensorBoard on port 6006 (pid 1236), started 0:57:36 ago. (Use '!kill 1236' to kill it.)